
## Import dependencies

In [1]:
import os
import time

import pandas
import torch

from proselflc.trainer.trainer_cnn_vision_derivedgrad import Trainer

import pprint
print = pprint.PrettyPrinter(indent=4).pprint

## Add params configurations

In [2]:
params = {}
params.update(
    {
        "data_name": "cifar100",
        "num_classes": 100,  # 1000
        "device": "gpu",
        #
        "num_workers": 8,
        #
        "counter": "iteration",
        "classes_per_batch": 8,
        #
        # To get deterministic results, please uncomment the line below
        # to set seed parameter.
        # "seed": 123,
    }
)

# data
params["symmetric_noise_rate"] = 0.4

# network
params["network_name"] = "resnet18"

# for the demo purpose, I set the total epochs to be small.
params["total_epochs"] = 100
params["eval_interval"] = 500 # iterations

# batch
params["batch_size"] = 128
params["sampler"] = "BalancedBatchSampler"
params["classes_per_batch"] = 64

# learning rate
params["lr"] = 0.2
params["weight_decay"] = 2e-3 # larger weight decay for resnet18
params["lr_scheduler"] = "WarmupMultiStepSchedule"
params["warmup_epochs"] = 0
params["milestones"] = [20000, 30000]

# optimisation
params["momentum"] = 0.9
params["batch_accumu_steps"] = 10
params["gamma"] = 0.1

# loss settings
params["loss_mode"] = "cross entropy"
params["trust_mode"] = "global*(1-H(p)/H(u))"
params["loss_name"] = "proselflc"
params["transit_time_ratio"] = 0.50
params["exp_base"] = 16
params["logit_soften_T"] = 0.5

print(params)

{   'batch_accumu_steps': 10,
    'batch_size': 128,
    'classes_per_batch': 64,
    'counter': 'iteration',
    'data_name': 'cifar100',
    'device': 'gpu',
    'eval_interval': 500,
    'exp_base': 16,
    'gamma': 0.1,
    'logit_soften_T': 0.5,
    'loss_mode': 'cross entropy',
    'loss_name': 'proselflc',
    'lr': 0.2,
    'lr_scheduler': 'WarmupMultiStepSchedule',
    'milestones': [20000, 30000],
    'momentum': 0.9,
    'network_name': 'resnet18',
    'num_classes': 100,
    'num_workers': 8,
    'sampler': 'BalancedBatchSampler',
    'symmetric_noise_rate': 0.4,
    'total_epochs': 100,
    'transit_time_ratio': 0.5,
    'trust_mode': 'global*(1-H(p)/H(u))',
    'warmup_epochs': 0,
    'weight_decay': 0.002}


## Create the folder to store intermediate and final results
* First, run `sudo mkdir /home/proselflc_experiments/ && sudo chmod -R 777 /home/proselflc_experiments/` so that so you have the write permission.
* Or set `WORK_DIR = "/home/your_username/proselflc_experiments/"`

In [3]:
WORK_DIR = "/home/proselflc_experiments/"
#
# use the time as a unique experiment identifier
dt_string = time.strftime("%Y%m%d-%H%M%S")
summary_writer_dir = (
    params["loss_name"]
    + "_"
    + dt_string
)
params["summary_writer_dir"] = (
    WORK_DIR
    + "/"
    + params["data_name"]
    + "_symmetric_noise_rate_"
    + str(params["symmetric_noise_rate"])
    + "/"
    + params["network_name"]
    + "/"
    + summary_writer_dir
)
if not os.path.exists(params["summary_writer_dir"]):
    os.makedirs(params["summary_writer_dir"])

## Init the trainer, and store the params configurations
For each experiment, we have one unique result folder to store the params configurations and learning curves.
Therefore, you can revisit any specific experiment whenever you need without losing any details.

In [4]:
trainer = Trainer(params=params)
# params["milestones"] was a list of integers, we convert it to a string before sinking.
params["milestones"] = str(params["milestones"])
dataframe = pandas.DataFrame(params, index=[0])
dataframe.to_csv(
    params["summary_writer_dir"] + "/params.csv",
    encoding="utf-8",
    index=False,
    sep="\t",
    mode="w",  #
)

Files already downloaded and verified
Files already downloaded and verified
Files already downloaded and verified


## Run the trainer and save the final model

In [5]:
trainer.train()
torch.save(
    trainer.network,
    params["summary_writer_dir"] + "/model.pt",
)
print(
    "The experiment is finished with details sinked in {}".format(
        params["summary_writer_dir"]
    )
)

Evaluating Network.....
Iteration= (500,2)/(39100, 100), lr=0.2000, batch_mean_epsilon=0.0001, valid batch size=128.0000, batch_mean_gtrust=0.0004, batch_mean_etrust=0.1624
clean_test: Loss= 4.1282, Accuracy= 0.0819, Entropy= 0.9285, Max_p= 0.0616,
noisy_subset: Loss= 4.8826, Accuracy= 0.0106, Entropy= 0.9404, Max_p= 0.0532,
clean_subset: Loss= 4.1465, Accuracy= 0.0855, Entropy= 0.9399, Max_p= 0.0537,
cleaned_noisy_subset: Loss= 4.1601, Accuracy= 0.0814, Entropy= 0.9402, Max_p= 0.0534,
Evaluating Network.....
Iteration= (1000,3)/(39100, 100), lr=0.2000, batch_mean_epsilon=0.0001, valid batch size=128.0000, batch_mean_gtrust=0.0005, batch_mean_etrust=0.2128
clean_test: Loss= 3.9089, Accuracy= 0.1116, Entropy= 0.9237, Max_p= 0.0665,
noisy_subset: Loss= 4.8363, Accuracy= 0.0104, Entropy= 0.9370, Max_p= 0.0569,
clean_subset: Loss= 3.9159, Accuracy= 0.1183, Entropy= 0.9359, Max_p= 0.0579,
cleaned_noisy_subset: Loss= 3.9486, Accuracy= 0.1108, Entropy= 0.9372, Max_p= 0.0567,


KeyboardInterrupt: 